In [ ]:
#--coding:utf-8--
#获得模型信息的代码

from keras.applications.resnet import ResNet50, preprocess_input
from keras_applications.resnext import ResNeXt50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = ResNeXt50(include_top=False,
                       weights=None,
                       input_shape=(224, 224, 3),
                       backend=keras.backend,
                       layers=keras.layers,
                       models=keras.models,
                       utils=keras.utils)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.summary()
print('the number of layers in this model:' + str(len(model.layers)))

In [ ]:
# --coding:utf-8--
import os
import sys
import glob
import matplotlib.pyplot as plt
import keras
from keras import __version__
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau


def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt


# 数据准备
print("导入数据，设置基础参数...")
IM_WIDTH, IM_HEIGHT = 100, 100  #densenet指定的图片尺寸
train_dir = '/kaggle/input/tomatolay/normal/train'  # 训练集数据路径
val_dir = '/kaggle/input/tomatolay/normal/val'  # 验证集数据
nb_classes = 10
nb_epoch = 20
batch_size = 64

nb_train_samples = get_nb_files(train_dir)  # 训练样本个数
nb_classes = len(glob.glob(train_dir + "/*"))  # 分类数
nb_val_samples = get_nb_files(val_dir)  #验证集样本个数
nb_epoch = int(nb_epoch)  # epoch数量
batch_size = int(batch_size)

#　图片增强
print("数据增强...")
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=30,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# 训练数据与测试数据
print("准备训练数据与测试数据...")
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(IM_WIDTH,
                                                                 IM_HEIGHT),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    class_mode='categorical')

class_dict = train_generator.class_indices
li = list(class_dict.keys())


# 添加新层
def add_new_last_layer(base_model, nb_classes):
    """
  添加最后的层
  输入
  base_model和分类数量
  输出
  新的keras的model
  """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(nb_classes,
                        activation='softmax')(x)  #new softmax layer
    model = Model(inputs=base_model.inputs, outputs=predictions)
    return model


#搭建模型
print("模型搭建中...")
model = InceptionResNetV2(weights='imagenet',
                          input_shape=(100, 100, 3),
                          include_top=False)
model = add_new_last_layer(model, nb_classes)
model.compile(optimizer=SGD(lr=0.001,
                            momentum=0.9,
                            decay=0.0001,
                            nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#设置回调
print("设置回调...")
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                            factor=0.1,
                                            patience=10,
                                            verbose=0,
                                            mode='auto',
                                            min_delta=0.0001,
                                            cooldown=0,
                                            min_lr=0)
filepath = "ResNeXt101.hdf5"
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
callbacks_list = [checkpoint, learning_rate_reduction]

#开始训练AD
print("开始训练")
history_ft = model.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples //
                                 batch_size,
                                 epochs=nb_epoch,
                                 validation_data=validation_generator,
                                 validation_steps=nb_val_samples // batch_size,
                                 callbacks=callbacks_list)

In [ ]:
test_dir= "/kaggle/input/tomatolay/normal/test"
test_generator_samples = get_nb_files(test_dir)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(IM_WIDTH,
                                                                 IM_HEIGHT),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                     shuffle = False)
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator_samples / batch_size)
print('test acc: %.4f%%' % test_acc)